In [1]:
import ipywidgets as widgets
from ipywidgets import HBox, VBox, Layout, IntProgress, Label
from IPython.display import display,clear_output

#from ipysheet import sheet, cell, row, column, cell_range, to_dataframe, from_dataframe, current
#import ipysheet

import pandas as pd
from tkinter import Tk, filedialog
import os, sys
import sqlalchemy as db
import logging
import matplotlib.pyplot as plt
import matplotlib.cbook as cbook
from matplotlib.patches import Rectangle

import qgrid
import glob

#from beakerx import *

import csv

from shutil import copyfile
from shutil import move

import re

from datetime import datetime

import ntpath

import misc

In [2]:
# project folder structure
#
# example_project/
# ├── crystal_images
# ├── crystal_screen
# ├── database
# └── workflow
#     ├── 1-inspect
#     ├── 2-soak
#     └── 3-mount
#     └── 4-mount-manual

In [3]:
#
# Notes
#

# qgrid prevents rendering with voila, but this can be circumvented, by
# - open /Applications/anaconda3/share/jupyter/nbextensions/qgrid/index.js
# - change: define(["@jupyter-widgets/base","base/js/dialog"],(function(__WEBPACK_EXTERNAL_MODULE__141__,...
#       to: define(["@jupyter-widgets/base"],(function(__WEBPACK_EXTERNAL_MODULE__141__,...
# - launch viola from command line: voila FragMAX_crystal_preparation.ipynb --enable_nbextensions=True
# more information can be found here:
# https://github.com/voila-dashboards/voila/issues/72

In [4]:
#
# folders
#

lp3_project_folder = os.path.join('/Users/tobkro/tmp/jupyter_test') # this needs to be hardcoded

global project_folder
project_folder = None

global db_dir
db_dir = None

global db_file
db_file = None

global crystal_image_folder
crystal_image_folder = None

global crystal_screen_folder
crystal_screen_folder = None

global workflow_folder
workflow_folder = None


#
# DB tables
#

global connection
connection = None

global engine
engine = None

global projectTable
projectTable = None

global crystalscreenTable
crystalscreenTable = None

global proteinTable
proteinTable = None

global crystal_plate_typeTable
crystal_plate_typeTable = None

global crystalplateTable
crystalplateTable = None

global markedcrystalTable
markedcrystalTable = None

global soakplateTable
soakplateTable = None

global compoundbatchTable
compoundbatchTable = None

global soakedcrystalTable
soakedcrystalTable = None

global mountedcrystalTable
mountedcrystalTable = None


#
# templates
#

db_template = os.path.join(os.getcwd(),'database', 'fragmax.sqlite')
crystal_plate_template = os.path.join(os.getcwd(),'templates','crystal_plate_template.csv')
shifter_template = os.path.join(os.getcwd(),'templates','shifter_template.csv')

In [5]:
# Resources
#
# https://towardsdatascience.com/sqlalchemy-python-tutorial-79a577141a91
# https://www3.ntu.edu.sg/home/ehchua/programming/sql/Relational_Database_Design.html
# https://kapernikov.com/ipywidgets-with-matplotlib/
# https://medium.com/@chadwick.balloo/qgrid-basics-5be2c278049e
# https://pandas.pydata.org/pandas-docs/stable/user_guide/merging.html
# https://jupyter.org/widgets
# https://nbviewer.jupyter.org/gist/anonymous/d7dbf536abade987157c085b1850416d
# https://nbviewer.jupyter.org/github/twosigma/beakerx/blob/master/StartHere.ipynb
# https://nbviewer.jupyter.org/github/twosigma/beakerx/blob/master/doc/python/TableAPI.ipynb



In [6]:
#
# logger
#

class OutputWidgetHandler(logging.Handler):
    """ Custom logging handler sending logs to an output widget """
    """ https://ipywidgets.readthedocs.io/en/latest/examples/Output%20Widget.html """

    def __init__(self, *args, **kwargs):
        super(OutputWidgetHandler, self).__init__(*args, **kwargs)
        layout = {
            'width': '100%',
            'height': '160px',
            'border': '1px solid black'
        }
        self.out = widgets.Output(layout=layout)

    def emit(self, record):
        """ Overload of logging.Handler method """
        formatted_record = self.format(record)
        new_output = {
            'name': 'stdout',
            'output_type': 'stream',
            'text': formatted_record+'\n'
        }
        self.out.outputs = (new_output, ) + self.out.outputs

    def show_logs(self):
        """ Show the logs """
        display(self.out)

    def clear_logs(self):
        """ Clear the current logs """
        self.out.clear_output()


In [7]:
#
# initialize logger
#

logger = logging.getLogger(__name__)
handler = OutputWidgetHandler()
handler.setFormatter(logging.Formatter('%(asctime)s - [%(levelname)s] %(message)s'))
logger.addHandler(handler)
logger.setLevel(logging.INFO)
logger.info('starting session')

In [8]:
#
# FragMAX header
#

file = open("images/FragMAX-logo-full.png", "rb")
image = file.read()
logo = widgets.Image(value=image, format='png', width=200, height=300)

#title = widgets.Label(value='Crystal Plate Preparation')
text = 'Crystal Preparation'
title = widgets.HTML(value = f"<b><font color='black'><font size = 10px>{text}</b>")

text = 'XX'
spacer = widgets.HTML(value = f"<b><font color='white'><font size = 6px>{text}</b>")

HBox(children=[logo, spacer, title],layout=Layout(width='100%', display='flex', align_items='center'))

In [9]:
#
# Functions for Project Description tab
#

def read_project(b):
    read_project_from_db()
    
def read_project_from_db():
    query = db.select([projectTable])
    ResultProxy = connection.execute(query)
    results = ResultProxy.fetchall()
    try:
        x = [dict(r) for r in results][0]
        project_name.value = str(x['Project_Name'])
        proposal_id.value = str(x['Proposal_ID'])
    except IndexError:
        logger.warning('no project name and proposal ID registered in DB')
        
    query = db.select([proteinTable])
    ResultProxy = connection.execute(query)
    results = ResultProxy.fetchall()
    try:
        x = [dict(r) for r in results][0]
        protein_name.value = str(x['Protein_Name'])
        protein_acronym.value = str(x['Protein_Acronym'])
    except IndexError:
        logger.warning('no protein name and protein acronym registered in DB')

    update_crystal_plate_widgets()
    
def save_project(b):
    query = db.select([projectTable.columns.Proposal_ID.distinct()])
    ResultProxy = connection.execute(query)
    existing_project_id = [x[0] for x in ResultProxy.fetchall()]
    
    logger.info('existing project_id: ' + str(existing_project_id))
    
    if existing_project_id == []:
        logger.info('adding project information: {0!s}, {1!s}'.format(project_name.value, proposal_id.value.replace(' ','')))
        values_list = [{
            'Project_Name': project_name.value,
            'Proposal_ID':  proposal_id.value.replace(' ','')
        }]  
        query = db.insert(projectTable) 
        ResultProxy = connection.execute(query,values_list)
    
    else:
        logger.info('updating project information: {0!s}, {1!s}'.format(project_name.value, proposal_id.value.replace(' ','')))
        query = db.update(projectTable).values(
            Project_Name = project_name.value,
            Proposal_ID = proposal_id.value.replace(' ','')).where(
            projectTable.columns.Proposal_ID == existing_project_id[0])
        results = connection.execute(query)
        
    query = db.select([proteinTable.columns.Protein_Acronym.distinct()])
    ResultProxy = connection.execute(query)
    existing_protein_acronym = [x[0] for x in ResultProxy.fetchall()]
    
    if existing_protein_acronym == []:
        values_list = [{
            'Protein_Name':    protein_name.value,
            'Protein_Acronym': protein_acronym.value.replace(' ','')
        }]  
        query = db.insert(proteinTable) 
        ResultProxy = connection.execute(query,values_list)
    
    else:
        query = db.update(proteinTable).values(
            Protein_Name = protein_name.value,
            Protein_Acronym = protein_acronym.value.replace(' ','')).where(
            proteinTable.columns.Protein_Acronym == existing_protein_acronym[0])
        results = connection.execute(query)

    update_crystal_plate_widgets()

def update_crystal_plate_widgets():
    query = db.select([proteinTable.columns.Protein_Acronym.distinct()])
    ResultProxy = connection.execute(query)
    existing_protein = [x[0] for x in ResultProxy.fetchall()]
    logger.info('found the following protein acronyms in database: ' + str(existing_protein))
    select_protein.options = existing_protein

    query = db.select([crystal_plate_typeTable.columns.Plate_Name.distinct()])
    ResultProxy = connection.execute(query)
    existing_plate_types = [x[0] for x in ResultProxy.fetchall()]
    logger.info('found the following crystal plate types in database: ' + str(existing_plate_types))
    select_plate_type.options = existing_plate_types

        
        
def default_folders(project_folder):
    global db_dir
    db_dir = os.path.join(project_folder,'database')
    if os.path.isdir(db_dir):
        logger.warning('DB folder exists: ' + db_dir)
    else:
        logger.info('creating DB folder: ' + db_dir)
        os.mkdir(db_dir)
        os.mkdir(os.path.join(db_dir, 'backup'))

    global crystal_image_folder
    crystal_image_folder = os.path.join(project_folder,'crystal_images')
    if os.path.isdir(crystal_image_folder):
        logger.warning('crystal image folder exists: ' + crystal_image_folder)
    else:
        logger.info('creating crystal image folder: ' + crystal_image_folder)
        os.mkdir(crystal_image_folder)

    global crystal_screen_folder
    crystal_screen_folder = os.path.join(project_folder,'crystal_screen')
    if os.path.isdir(crystal_screen_folder):
        logger.warning('crystal screen folder exists: ' + crystal_screen_folder)
    else:
        logger.info('creating screen image folder: ' + crystal_screen_folder)
        os.mkdir(crystal_screen_folder)

    global workflow_folder
    workflow_folder = os.path.join(project_folder,'workflow')
    if os.path.isdir(workflow_folder):
        logger.warning('workflow folder exists: ' + workflow_folder)
    else:
        logger.info('creating workflow folder: ' + workflow_folder)
        os.mkdir(workflow_folder)

    workflow_inspect_folder = os.path.join(project_folder, 'workflow', '1-inspect')
    if os.path.isdir(workflow_inspect_folder):
        logger.warning('workflow/1-inspect folder exists: ' + workflow_inspect_folder)
    else:
        logger.info('creating workflow/1-inspect folder: ' + workflow_inspect_folder)
        os.mkdir(workflow_inspect_folder)
        os.mkdir(os.path.join(workflow_inspect_folder, 'backup'))

    workflow_soak_folder = os.path.join(project_folder, 'workflow', '2-soak')
    if os.path.isdir(workflow_soak_folder):
        logger.warning('workflow/2-soak folder exists: ' + workflow_soak_folder)
    else:
        logger.info('creating workflow/2-soak folder: ' + workflow_soak_folder)
        os.mkdir(workflow_soak_folder)
        os.mkdir(os.path.join(workflow_soak_folder, 'backup'))

    workflow_mount_folder = os.path.join(project_folder, 'workflow', '3-mount')
    if os.path.isdir(workflow_mount_folder):
        logger.warning('workflow/3-mount folder exists: ' + workflow_mount_folder)
    else:
        logger.info('creating workflow/3-mount folder: ' + workflow_mount_folder)
        os.mkdir(workflow_mount_folder)
        os.mkdir(os.path.join(workflow_mount_folder, 'backup'))

    workflow_mount_manual_folder = os.path.join(project_folder, 'workflow', '4-mount-manual')
    if os.path.isdir(workflow_mount_manual_folder):
        logger.warning('workflow/4-mount-manual folder exists: ' + workflow_mount_manual_folder)
    else:
        logger.info('creating workflow/4-mount-manual folder: ' + workflow_mount_manual_folder)
        os.mkdir(workflow_mount_manual_folder)
        os.mkdir(os.path.join(workflow_mount_manual_folder, 'backup'))

def set_db(project_folder):
    global db_file
    db_file = os.path.join(project_folder,'database', 'fragmax.sqlite')
    if os.path.isfile(db_file):
        logger.info('found DB file: ' + db_file)
        backup_db()
    else:
        logger.warning('DB file does not exist: ' + db_file)
        logger.info('copying DB template file from ' + db_template + ' to ' + db_file)
        copyfile(db_template,db_file)
    init_db()

def backup_db():
    now = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
    logger.info('creating backup current DB file: ' + os.path.join(db_dir, 'backup', 'fragmax.sqlite.' + now))
    copyfile(db_file,os.path.join(db_dir, 'backup', 'fragmax.sqlite.' + now))
    
def init_db():
    logger.info('initializing DB...')
    global engine
    engine = db.create_engine('sqlite:///' + db_file)
    global connection
    connection = engine.connect()
    metadata = db.MetaData()

    global projectTable
    projectTable = db.Table('Project', metadata, autoload=True, autoload_with=engine)

    global crystalscreenTable
    crystalscreenTable = db.Table('CrystalScreen', metadata, autoload=True, autoload_with=engine)
    
    global proteinTable
    proteinTable = db.Table('Protein', metadata, autoload=True, autoload_with=engine)

    global crystal_plate_typeTable
    crystal_plate_typeTable = db.Table('CrystalPlateType', metadata, autoload=True, autoload_with=engine)
    
    global crystalplateTable
    crystalplateTable = db.Table('CrystalPlate', metadata, autoload=True, autoload_with=engine)
    
    global markedcrystalTable
    markedcrystalTable = db.Table('MarkedCrystals', metadata, autoload=True, autoload_with=engine)
    
    global soakplateTable
    soakplateTable = db.Table('SoakPlate', metadata, autoload=True, autoload_with=engine)
    
    global compoundbatchTable
    compoundbatchTable = db.Table('CompoundBatchTable', metadata, autoload=True, autoload_with=engine)
    
    global soakedcrystalTable
    soakedcrystalTable = db.Table('SoakedCrystals', metadata, autoload=True, autoload_with=engine)

    global mountedcrystalTable
    mountedcrystalTable = db.Table('MountedCrystals', metadata, autoload=True, autoload_with=engine)

    logger.info('finished initializing DB')
    
    
def select_project_directory(b):
    clear_output()                                         # Button is deleted after it is clicked.
    root = Tk()
    root.withdraw()                                        # Hide the main window.
    root.call('wm', 'attributes', '.', '-topmost', True)   # Raise the root to the top of all windows.
    b.folder = filedialog.askdirectory(initialdir=lp3_project_folder, title="Select project directory")   

    if os.path.isdir(b.folder):
        project_folder = b.folder
        default_folders(project_folder)
        set_db(project_folder)
        project_directory.value = str(project_folder)
        read_project_from_db()
        tab0.add_class("all_good")
    else:
        logger.error('selected project folder does not exist: ' + str(b.folder))
        


In [10]:
#
# Project Description tab
#

g0 = widgets.GridspecLayout(6, 4)

g0[0,0] = Label("Project Name", layout=Layout(display="flex", justify_content="center"))
project_name = widgets.Text(value='', layout=widgets.Layout(height="auto", width="200"))
g0[0,1:] = project_name

g0[1,0] = Label("Proposal ID", layout=Layout(display="flex", justify_content="center"))
proposal_id = widgets.Text(value='', layout=widgets.Layout(height="auto", width="100"))
g0[1,1:] = proposal_id

g0[2,0] = Label("Protein Name", layout=Layout(display="flex", justify_content="center"))
protein_name = widgets.Text(value='', layout=widgets.Layout(height="auto", width="100"))
g0[2,1:] = protein_name

g0[3,0] = Label("Protein Acronym", layout=Layout(display="flex", justify_content="center"))
protein_acronym = widgets.Text(value='', layout=widgets.Layout(height="auto", width="100"))
g0[3,1:] = protein_acronym

#g0[4,0] = Label("Protein UniProt ID", layout=Layout(display="flex", justify_content="center"))
#protein_acronym = widgets.Text(value='', layout=widgets.Layout(height="auto", width="100"))
#g0[4,1:] = protein_acronym

#g0[5,0] = Label("Protein Sequence", layout=Layout(display="flex", justify_content="center"))
#protein_acronym = widgets.Text(value='', layout=widgets.Layout(height="auto", width="100"))
#g0[5,1:] = protein_acronym

#g0[6,0] = Label("Mutations (e.g. V26A D47N)", layout=Layout(display="flex", justify_content="center"))
#protein_acronym = widgets.Text(value='', layout=widgets.Layout(height="auto", width="100"))
#g0[6,1:] = protein_acronym

#g0[7,0] = Label("Source Organism)", layout=Layout(display="flex", justify_content="center"))
#protein_acronym = widgets.Text(value='', layout=widgets.Layout(height="auto", width="100"))
#g0[7,1:] = protein_acronym

#g0[8,0] = Label("Recombinant?)", layout=Layout(display="flex", justify_content="center"))
#protein_acronym = widgets.Text(value='', layout=widgets.Layout(height="auto", width="100"))
#g0[8,1:] = protein_acronym

#g0[9,0] = Label("Expression Host)", layout=Layout(display="flex", justify_content="center"))
#protein_acronym = widgets.Text(value='', layout=widgets.Layout(height="auto", width="100"))
#g0[9,1:] = protein_acronym

#g0[10,0] = Label("Expression Strain)", layout=Layout(display="flex", justify_content="center"))
#protein_acronym = widgets.Text(value='', layout=widgets.Layout(height="auto", width="100"))
#g0[10,1:] = protein_acronym

#g0[11,0] = Label("Expression Vectortype)", layout=Layout(display="flex", justify_content="center"))
#protein_acronym = widgets.Text(value='', layout=widgets.Layout(height="auto", width="100"))
#g0[11,1:] = protein_acronym

#g0[12,0] = Label("Expression Vectorname)", layout=Layout(display="flex", justify_content="center"))
#protein_acronym = widgets.Text(value='', layout=widgets.Layout(height="auto", width="100"))
#g0[12,1:] = protein_acronym



g0[4,0] = Label("Project Directory", layout=Layout(display="flex", justify_content="center"))
project_directory = widgets.Text(value='', layout=widgets.Layout(height="auto", width="200"))
g0[4,1:2] = project_directory
select_project_directory_button = widgets.Button(description='Select', 
                                                 style= {'button_color':'green'}, tooltip=misc.select_project_directory_button_tip())
select_project_directory_button.on_click(select_project_directory)
g0[4,3] = select_project_directory_button


read_project_button = widgets.Button(description='Read from DB')
read_project_button.on_click(read_project)
g0[5,0] = read_project_button

save_project_button = widgets.Button(description='Save to DB')
save_project_button.on_click(save_project)
g0[5,1] = save_project_button



In [11]:
#
# Functions for Crystal Screen tab
#

def upload_screen_csv(b):
    clear_output()                                         # Button is deleted after it is clicked.
    root = Tk()
    root.withdraw()                                        # Hide the main window.
    root.call('wm', 'attributes', '.', '-topmost', True)   # Raise the root to the top of all windows.
    b.files = filedialog.askopenfilename(multiple=True)    # List of selected files will be set button's file attribute.
    if os.path.isfile(b.files[0]):
        logger.info('loading ' + b.files[0])
        screen_name.value = ntpath.basename(b.files[0]).split('.')[0]
        add_screen_to_dropdown()
        df = pd.read_csv(b.files[0], sep=';')
        screen_sheet.df = df
    else:
        logger.error('cannot read file ' + b.files[0])

def add_screen(arg):
    add_screen_to_dropdown()
    
def add_screen_to_dropdown():
    l = []
    for opt in select_screen.options: l.append(opt)
    if screen_name.value not in l:
        logger.info('adding ' + screen_name.value + ' to screen dropdown')
        l.append(screen_name.value)
    else:
        logger.warning(screen_name.value + ' exists in screen dropdown')
    select_screen.options = l

def save_screen_to_db(arg):
#    df = to_dataframe(screen_sheet)
#    df = screen_sheet.df
    df = screen_sheet.get_changed_df()
    CrystalScreen_Name = select_screen.value.replace(' ','')
    logger.info('saving ' + CrystalScreen_Name + ' crystal screen to database')

    query = db.select([crystalscreenTable.columns.CrystalScreen_Name.distinct()])
    ResultProxy = connection.execute(query)
    existing_crystalscreens = [x[0] for x in ResultProxy.fetchall()]

    for index, row in df.iterrows():
        crystal_screen_progress.value = index
        condition = df.at[index,'CrystalScreen_Condition']
        well = df.at[index,'CrystalScreen_Well']
        screen_id = CrystalScreen_Name + '-' + well
        if CrystalScreen_Name in existing_crystalscreens:
            logger.warning('crystal screen exists in database; updating records: {0!s} - {1!s}'.format(well, condition))
            query = db.update(crystalscreenTable).values(
                CrystalScreen_Condition = condition).where(
                crystalscreenTable.columns.CrystalScreen_ID == screen_id)
            results = connection.execute(query)
        else:
            logger.info('crystal screen does not exist in database; inserting records: {0!s} - {1!s}'.format(well, condition))
            values_list = [{
                'CrystalScreen_Condition': condition,
                'CrystalScreen_Well':      well,
                'CrystalScreen_Name':      CrystalScreen_Name,
                'CrystalScreen_ID':        screen_id
            }]  
            query = db.insert(crystalscreenTable) 
            ResultProxy = connection.execute(query,values_list)
        crystal_screen_progress.value = 0

def refresh_screen_dropdown(arg):
    query = db.select([crystalscreenTable.columns.CrystalScreen_Name.distinct()])
    ResultProxy = connection.execute(query)
    existing_crystalscreens = [x[0] for x in ResultProxy.fetchall()]
    select_screen.options = existing_crystalscreens
    logger.info('updating screen selection dropdown: ' + str(existing_crystalscreens))

    
def load_screen_from_db(arg):
    query = db.select([crystalscreenTable.columns.CrystalScreen_Name.distinct()])
    ResultProxy = connection.execute(query)
    existing_crystalscreens = [x[0] for x in ResultProxy.fetchall()]
    if select_screen.value in existing_crystalscreens:
        logger.info('screen ' + select_screen.value + ' exists in database')
        query = db.select([crystalscreenTable.columns.CrystalScreen_Well,
                          crystalscreenTable.columns.CrystalScreen_Condition]).where(
                          crystalscreenTable.columns.CrystalScreen_Name == select_screen.value)
        ResultProxy = connection.execute(query)
        result = ResultProxy.fetchall()
        logger.info('loading information for screen ' + select_screen.value + ' from database')
        df = pd.read_sql_query(query, engine)
        screen_sheet.df = df
    else:
        logger.warning('screen ' + select_screen.value + ' does not exist in database; skipping...')

def save_screen_csv(b):
    CrystalScreen_Name = select_screen.value.replace(' ','')
    logger.warning('removing whitespaces from crystal screen name: ' + CrystalScreen_Name)
    logger.info('trying to copy empty crystal screen CSV template with name ' + CrystalScreen_Name + ' to ' + crystal_screen_folder)
    if os.path.isfile(os.path.join(crystal_screen_folder, CrystalScreen_Name + '.csv')):
        logger.error('file exists in ' + os.path.join(crystal_screen_folder, CrystalScreen_Name + '.csv'))
    else:
        logger.info('creating new template ' + os.path.join(crystal_screen_folder, CrystalScreen_Name + '.csv'))
        copyfile(crystal_plate_template,os.path.join(crystal_screen_folder, CrystalScreen_Name + '.csv'))

def save_dragonfly_to_csv(dragonflyFile):
    csv = 'CrystalScreen_Well,CrystalScreen_Condition\n'
    new_condition = False
    for line in open(dragonflyFile):

        if new_condition and line.replace('\n','') == '':
            csv += well + ',' + condition[:-3] + '\n'
            new_condition = False

        if new_condition:
            condition += line.replace('\n','').replace(',','.') + ' - '

        if line.endswith(':\n') and not 'Components' in line:
            well = line.replace(':\n','')
            if len(well) == 2:
                well = well[0] + '0' + well[1]
            new_condition = True
            condition = ''
    logger.info('saving dragonfly txt file as csv: ' + dragonflyFile.replace('.txt','.csv'))
    f = open(dragonflyFile.replace('.txt','.csv'), 'w')
    f.write(csv)
    f.close

        
def import_dragonfly(b):
    clear_output()                                         # Button is deleted after it is clicked.
    root = Tk()
    root.withdraw()                                        # Hide the main window.
    root.call('wm', 'attributes', '.', '-topmost', True)   # Raise the root to the top of all windows.
    b.files = filedialog.askopenfilename(multiple=True,
                                         initialdir=crystal_screen_folder, 
                                         title="Select dragonfly file",
                                         filetypes=[("Text Files", "*.txt")])    # List of selected files will be set button's file attribute.
    
    if os.path.isfile(b.files[0]):
        screen_name.value = ntpath.basename(b.files[0]).split('.')[0]
        add_screen_to_dropdown()
        save_dragonfly_to_csv(b.files[0])
        dragonflyCSV = b.files[0].replace('.txt','.csv')
        logger.info('loading ' + dragonflyCSV)
        df = pd.read_csv(dragonflyCSV, sep=',')
        screen_sheet.df = df


In [12]:
#
# Crystal Screen tab
#


g1 = widgets.GridspecLayout(3, 4)

#g1[0,0] = widgets.Button(description="Enter New Screen Name", 
#                         layout=widgets.Layout(height="auto", width="auto"), style = {'button_color':'white'})

g1[0,0] = Label("Enter New Screen Name", layout=Layout(display="flex", justify_content="center"))

screen_name = widgets.Text(value='', layout=widgets.Layout(height="auto", width="auto"))
g1[0,1] = screen_name
add_screen_button = widgets.Button(description='Add', tooltip = misc.add_screen_button_tip())
add_screen_button.on_click(add_screen)
g1[0,2] = add_screen_button
#g1[0,3] = widgets.Button(description="Note: spaces will be removed", layout=auto_hw, style=white_button)

#g1[1,0] = widgets.Button(description="Select Screen", 
#                         layout=widgets.Layout(height="auto", width="auto"), style={'button_color':'white'})
g1[1,0] = Label("Select Screen", layout=Layout(display="flex", justify_content="center"))
select_screen = widgets.Dropdown()
g1[1,1] = select_screen

refresh_screen_button = widgets.Button(description='Refresh Screen List', tooltip = misc.refresh_screen_button_tip())
refresh_screen_button.on_click(refresh_screen_dropdown)
g1[1,2] = refresh_screen_button

load_selected_screen_button = widgets.Button(description="Load Screen from DB", 
                                             layout=widgets.Layout(height="auto", width="auto"), 
                                             tooltip = misc.load_selected_screen_button_tip())
load_selected_screen_button.on_click(load_screen_from_db)
g1[2,0] = load_selected_screen_button
           
                                     
#save_screen_csv_button = widgets.Button(description="Save CSV template", 
#                                             layout=widgets.Layout(height="auto", width="auto"), 
#                                             tooltip = misc.save_screen_csv_button_tip(str(crystal_screen_folder), 
#                                                                        str(select_screen.value).replace(' ','').replace('None',''))())
save_screen_csv_button = widgets.Button(description="Save CSV template", 
                                             layout=widgets.Layout(height="auto", width="auto"), 
                                             tooltip = misc.save_screen_csv_button_tip(str(crystal_screen_folder),
                                                                                       str(select_screen.value)))
save_screen_csv_button.on_click(save_screen_csv)
g1[2,1] = save_screen_csv_button
                                     
                                     
upload_screen_csv_button = widgets.Button(description="Upload CSV file", 
                                          layout=widgets.Layout(height="auto", width="auto"))
upload_screen_csv_button.on_click(upload_screen_csv)
g1[2,2] = upload_screen_csv_button

import_dragonfly_button = widgets.Button(description="Import Dragonfly file", 
                                          layout=widgets.Layout(height="auto", width="auto"))
import_dragonfly_button.on_click(import_dragonfly)
g1[2,3] = import_dragonfly_button

#
# ipysheet
#

#screen_sheet = sheet(rows=96, columns=2, column_headers=['CrystalScreen_Well','CrystalScreen_Condition'])
#screen_sheet.layout.height = '350px'   # adjust to monitor size
df_template = pd.read_csv(crystal_plate_template)
#screen_sheet = qgrid.show_grid(df, grid_options={'sortable': False})
#grid_options={'forceFitColumns': False, 'defaultColumnWidth': 100}
screen_sheet = qgrid.QgridWidget(df=df_template, show_toolbar=False)


save_screen_to_db_button = widgets.Button(description='Save CrystalScreen to Database', 
                    layout=widgets.Layout(height="auto", width="auto"), style={'button_color':'gray'})
save_screen_to_db_button.on_click(save_screen_to_db)

crystal_screen_progress = IntProgress(min=0, max=95)
                                                                          

In [13]:
#
# Functions for Crystal Plate tab
#

def add_plate_barcode(arg):
    l = []
    for opt in select_barcode.options: l.append(opt)
    if plate_barcode.value not in l:
        logger.info('adding ' + plate_barcode.value + ' to barcode dropdown')
        l.append(plate_barcode.value)
    else:
        logger.warning(plate_barcode.value + ' exists in screen dropdown')
    select_barcode.options = l

def refresh_screens(arg):
    query = db.select([crystalscreenTable.columns.CrystalScreen_Name.distinct()])
    ResultProxy = connection.execute(query)
    existing_crystalscreen = [x[0] for x in ResultProxy.fetchall()]
    select_screen_for_plate.options = existing_crystalscreen
    logger.info('updating screen selection dropdown: ' + str(existing_crystalscreen))

def refresh_barcode(arg):
    query = db.select([crystalplateTable.columns.CrystalPlate_Barcode.distinct()])
    ResultProxy = connection.execute(query)
    existing_crystalplates = [x[0] for x in ResultProxy.fetchall()]
    select_barcode.options = existing_crystalplates
    logger.info('updating barcode selection dropdown: ' + str(existing_crystalplates))

def save_plate_to_db(arg):
    logger.info('saving information for ' + select_barcode.value + ' to database')
    query = db.select([crystalplateTable.columns.CrystalPlate_Barcode.distinct()])
    ResultProxy = connection.execute(query)
    existing_barcodes = [x[0] for x in ResultProxy.fetchall()]   

    try:
        _protein_concentration = float(protein_concentration.value)
    except ValueError:
        _protein_concentration = 0.0
    
    try:
        _temperature = float(temperature.value)
    except ValueError:
        _temperature = 0.0
        
    try:
        _reservoir_volume = float(reservoir_volume.value)
    except ValueError:
        _reservoir_volume = 0.0
               
    try:
        _subwell_a_protein = float(subwell_a_protein.value)
    except ValueError:
        _subwell_a_protein = 0.0

    try:
        _subwell_a_reservoir = float(subwell_a_reservoir.value)
    except ValueError:
        _subwell_a_reservoir = 0.0

    try:
        _subwell_a_seed = float(subwell_a_seed.value)
    except ValueError:
        _subwell_a_seed = 0.0

    try:
        _subwell_c_protein = float(subwell_c_protein.value)
    except ValueError:
        _subwell_c_protein = 0.0

    try:
        _subwell_c_reservoir = float(subwell_c_reservoir.value)
    except ValueError:
        _subwell_c_reservoir = 0.0

    try:
        _subwell_c_seed = float(subwell_c_seed.value)
    except ValueError:
        _subwell_c_seed = 0.0

    try:
        _subwell_d_protein = float(subwell_d_protein.value)
    except ValueError:
        _subwell_d_protein = 0.0

    try:
        _subwell_d_reservoir = float(subwell_d_reservoir.value)
    except ValueError:
        _subwell_d_reservoir = 0.0

    try:
        _subwell_d_seed = float(subwell_d_seed.value)
    except ValueError:
        _subwell_d_seed = 0.0
    
    if select_barcode.value in existing_barcodes:
        logger.warning('plate barcode ' + select_barcode.value + ' exists in database; updating records...')
        query = db.update(crystalplateTable).values(
            Protein_Acronym = select_protein.value,
            Protein_Concentration = _protein_concentration,
            CrystalScreen_Name = select_screen_for_plate.value,
            Protein_Buffer = protein_buffer.value,
            Temperature = _temperature,
            Plate_Type = select_plate_type.value,
            Reservoir_Volume = _reservoir_volume,
            Subwell_A_Vol_Protein = _subwell_a_protein,
            Subwell_A_Vol_Reservoir = _subwell_a_reservoir,
            Subwell_A_Vol_Seed = _subwell_a_seed,
            Subwell_C_Vol_Protein = _subwell_c_protein,
            Subwell_C_Vol_Reservoir = _subwell_c_reservoir,
            Subwell_C_Vol_Seed = _subwell_c_seed,
            Subwell_D_Vol_Protein = _subwell_d_protein,
            Subwell_D_Vol_Reservoir = _subwell_d_reservoir,
            Subwell_D_Vol_Seed = _subwell_d_seed
        ).where(crystalplateTable.columns.CrystalPlate_Barcode == select_barcode.value)
        results = connection.execute(query)
    else:
        logger.info('plate barcode ' + select_barcode.value + ' does not exist in database; inserting records...')
        values_list = [{
            'CrystalPlate_Barcode':    select_barcode.value,
            'Protein_Acronym':         select_protein.value,
            'Protein_Concentration':   _protein_concentration,
            'CrystalScreen_Name':      select_screen_for_plate.value,
            'Protein_Buffer':          protein_buffer.value,
            'Temperature':             _temperature,
            'Plate_Type':              select_plate_type.value,
            'Reservoir_Volume':        _reservoir_volume,
            'Subwell_A_Vol_Protein':   _subwell_a_protein,
            'Subwell_A_Vol_Reservoir': _subwell_a_reservoir,
            'Subwell_A_Vol_Seed':      _subwell_a_seed,
            'Subwell_C_Vol_Protein':   _subwell_c_protein,
            'Subwell_C_Vol_Reservoir': _subwell_c_reservoir,
            'Subwell_C_Vol_Seed':      _subwell_c_seed,
            'Subwell_D_Vol_Protein':   _subwell_d_protein,
            'Subwell_D_Vol_Reservoir': _subwell_d_reservoir,
            'Subwell_D_Vol_Seed':      _subwell_d_seed
              }]  
        query = db.insert(crystalplateTable) 
        ResultProxy = connection.execute(query,values_list)

    save_shifter_csv_to_inspect_folder(_subwell_a_protein, _subwell_c_protein, _subwell_d_protein, select_barcode.value, select_plate_type.value)

        
#    if os.path.isfile(os.path.join(workflow_folder,'1-inspect',select_barcode.value + '.csv')):
#        logger.warning('file exists in ' + os.path.join(workflow_folder,'1-inspect',select_barcode.value + '.csv'))
#    else:
#        logger.info('creating new template ' + os.path.join(workflow_folder,'1-inspect',select_barcode.value + '.csv'))
#        copyfile(shifter_template,os.path.join(workflow_folder,'1-inspect',select_barcode.value + '.csv'))

    #    if os.path.isfile(os.path.join(workflow_folder,'mounting',select_barcode.value + '.csv')):
#        logger.warning('file exists in ' + os.path.join(workflow_folder,'mounting',select_barcode.value + '.csv'))
#    else:
#        logger.info('creating new template ' + os.path.join(workflow_folder,'mounting',select_barcode.value + '.csv'))
#        copyfile(shifter_template,os.path.join(workflow_folder,'mounting',select_barcode.value + '.csv'))

def save_shifter_csv_to_inspect_folder(subwell_a, subwell_c, subwell_d, barcode, plate_type):
    # make backup of file exisits
    logger.info('writing CSV file for shifter inspection as ' + os.path.join(workflow_folder, '1-inspect', barcode + '.csv'))
    if os.path.isfile(os.path.join(workflow_folder, '1-inspect', barcode + '.csv')):
        logger.warning('file exists ' + os.path.join(workflow_folder, '1-inspect', barcode + '.csv'))
        now = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
        logger.info('backing up exisiting file as ' + barcode + '.csv.' + now)
        move(os.path.join(workflow_folder, '1-inspect', barcode + '.csv'),os.path.join(workflow_folder, '1-inspect', barcode + '.csv.' + now))        
    csv = ''
    subwells = []
    if int(subwell_a) != 0:
        subwells.append('a')
    if int(subwell_c) != 0:
        subwells.append('c')
    if int(subwell_d) != 0:
        subwells.append('d')
    rows = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H']
    columns = ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12']
    for row in rows:
        for column in columns:
            for subwell in subwells:
                csv += '{0!s},{1!s},AM,{2!s},{3!s},{4!s},,,,,,,,,\n'.format(plate_type, barcode, row, column, subwell)
    f = open(os.path.join(workflow_folder, '1-inspect', barcode + '.csv'), 'w')
    f.write(csv)
    f.close()
                

def load_plate_from_db(arg):
    logger.info('loading information for crystal plate ' + select_barcode.value)
    query = db.select([crystalplateTable]).where(
        crystalplateTable.columns.CrystalPlate_Barcode == select_barcode.value)
    ResultProxy = connection.execute(query)
    plate = ResultProxy.fetchall()
    x = [dict(r) for r in plate][0]
    reservoir_volume.value = str(x['Reservoir_Volume'])
    protein_concentration.value = str(x['Protein_Concentration'])
    temperature.value = str(x['Temperature'])
    subwell_a_protein.value = str(x['Subwell_A_Vol_Protein'])
    subwell_a_reservoir.value = str(x['Subwell_A_Vol_Reservoir'])
    subwell_a_seed.value = str(x['Subwell_A_Vol_Seed'])
    subwell_c_protein.value = str(x['Subwell_C_Vol_Protein'])
    subwell_c_reservoir.value = str(x['Subwell_C_Vol_Reservoir'])
    subwell_c_seed.value = str(x['Subwell_C_Vol_Seed'])
    subwell_d_protein.value = str(x['Subwell_D_Vol_Protein'])
    subwell_d_reservoir.value = str(x['Subwell_D_Vol_Reservoir'])
    subwell_d_seed.value = str(x['Subwell_D_Vol_Seed'])
    protein_buffer.value = str(x['Protein_Buffer'])
    crystalscreen = str(x['CrystalScreen_Name'])

    query = db.select([crystalscreenTable.columns.CrystalScreen_Name.distinct()])
    ResultProxy = connection.execute(query)
    existing_crystalscreen = [x[0] for x in ResultProxy.fetchall()]
    select_screen_for_plate.options = existing_crystalscreen
    select_screen_for_plate.value = crystalscreen

    

In [14]:
#
# Crystal Plate tab
#

g21 = widgets.GridspecLayout(10, 4)

#g21[0,0] = widgets.Button(description="Enter New Barcode", 
#                           layout=widgets.Layout(height="auto", width="auto"), style= {'button_color':'white'})
g21[0,0] = Label("Enter New Barcode", layout=Layout(display="flex", justify_content="center"))

plate_barcode = widgets.Text(value='', layout=widgets.Layout(height="auto", width="100"))
g21[0,1] = plate_barcode
add_plate_barcode_button = widgets.Button(description='Add')
add_plate_barcode_button.on_click(add_plate_barcode)
g21[0,2] = add_plate_barcode_button

#g21[1,0] = widgets.Button(description="Select Barcode", 
#                           layout=widgets.Layout(height="auto", width="auto"), style= {'button_color':'white'})
g21[1,0] = Label("Select Barcode", layout=Layout(display="flex", justify_content="center"))
select_barcode = widgets.Dropdown()
g21[1,1] = select_barcode
refresh_barcode_button = widgets.Button(description='Refresh Barcode List')
refresh_barcode_button.on_click(refresh_barcode)
g21[1,2] = refresh_barcode_button

#g21[2,0] = widgets.Button(description="Screen", 
#                           layout=widgets.Layout(height="auto", width="auto"), style= {'button_color':'white'})
g21[2,0] = Label("Screen", layout=Layout(display="flex", justify_content="center"))
select_screen_for_plate = widgets.Dropdown()
g21[2,1] = select_screen_for_plate
refresh_screens_button = widgets.Button(description='Refresh Screen List')
refresh_screens_button.on_click(refresh_screens)
g21[2,2] = refresh_screens_button


load_plate_from_db_button = widgets.Button(description='Load Plate from DB', style= {'button_color':'orange'})
load_plate_from_db_button.on_click(load_plate_from_db)
g21[4:5,2] = load_plate_from_db_button



#g21[3,0] = widgets.Button(description="Protein Concentration (mg/ml)",
#                           layout=widgets.Layout(height="auto", width="auto"), style= {'button_color':'white'})
g21[3,0] = Label("Protein Concentration (mg/ml)", layout=Layout(display="flex", justify_content="center"))
protein_concentration = widgets.Text(value='', layout=widgets.Layout(height="auto", width="100"))
g21[3,1] = protein_concentration

#g21[4,0] = widgets.Button(description="Protein", 
#                           layout=widgets.Layout(height="auto", width="auto"), style= {'button_color':'white'})
g21[4,0] = Label("Protein", layout=Layout(display="flex", justify_content="center"))
select_protein = widgets.Dropdown()
g21[4,1] = select_protein

#g21[5,0] = widgets.Button(description="Temperature (K)", 
#                           layout=widgets.Layout(height="auto", width="auto"),style= {'button_color':'white'})
g21[5,0] = Label("Temperature (K)", layout=Layout(display="flex", justify_content="center"))
temperature = widgets.Text(value='', layout=widgets.Layout(height="auto", width="100"))
g21[5,1] = temperature

#g21[6,0] = widgets.Button(description="Protein Buffer",
#                           layout=widgets.Layout(height="auto", width="auto"), style= {'button_color':'white'})
g21[6,0] = Label("Protein Buffer", layout=Layout(display="flex", justify_content="center"))
protein_buffer = widgets.Text(value='', layout=widgets.Layout(height="auto", width="100"))
g21[6,1] = protein_buffer

#g21[7,0] = widgets.Button(description="Reservoir Volume",
#                           layout=widgets.Layout(height="auto", width="auto"), style= {'button_color':'white'})
g21[7,0] = Label("Reservoir Volume (\u03BCL)", layout=Layout(display="flex", justify_content="center"))
reservoir_volume = widgets.Text(value='', layout=widgets.Layout(height="auto", width="100"))
g21[7,1] = reservoir_volume

#g21[8,0] = widgets.Button(description="Plate Type", 
#                           layout=widgets.Layout(height="auto", width="auto"), style= {'button_color':'white'})
g21[8,0] = Label("Plate Type", layout=Layout(display="flex", justify_content="center"))
select_plate_type = widgets.Dropdown()
g21[8,1] = select_plate_type


file = open("images/swiss_ci_layout.png", "rb")
image = file.read()
swissci = widgets.Image(value=image, format='png', width=200, height=200)
g21[1:-1,-1] = swissci


g22 = widgets.GridspecLayout(11, 3)

g22[0,0:] = widgets.Button(description="subwell a", layout=widgets.Layout(height="auto", width="auto"), 
                            button_style="info")
#g22[0,0:] = Label("subwell a", layout=Layout(display="flex", justify_content="center"))
g22[1,0] = widgets.Button(description="V(Protein) [nL]", layout=widgets.Layout(height="auto", width="auto"), 
                            button_style="info")
g22[1,1] = widgets.Button(description="V(Reservoir) [nL]", layout=widgets.Layout(height="auto", width="auto"), 
                            button_style="info")
g22[1,2] = widgets.Button(description="V(Seed) [nL]", layout=widgets.Layout(height="auto", width="auto"), 
                            button_style="info")
subwell_a_protein = widgets.Text(layout=widgets.Layout(height="auto", width="100"))
g22[2,0] = subwell_a_protein
subwell_a_reservoir = widgets.Text(layout=widgets.Layout(height="auto", width="100"))
g22[2,1] = subwell_a_reservoir
subwell_a_seed = widgets.Text(layout=widgets.Layout(height="auto", width="100"))
g22[2,2] = subwell_a_seed

g22[3,0:] = widgets.Button(description="subwell c", layout=widgets.Layout(height="auto", width="auto"), 
                            button_style="primary")
g22[4,0] = widgets.Button(description="V(Protein) [nL]", layout=widgets.Layout(height="auto", width="auto"), 
                            button_style="primary")
g22[4,1] = widgets.Button(description="V(Reservoir) [nL]", layout=widgets.Layout(height="auto", width="auto"), 
                            button_style="primary")
g22[4,2] = widgets.Button(description="V(Seed) [nL]", layout=widgets.Layout(height="auto", width="auto"), 
                            button_style="primary")
subwell_c_protein = widgets.Text(layout=widgets.Layout(height="auto", width="100"))
g22[5,0] = subwell_c_protein
subwell_c_reservoir = widgets.Text(layout=widgets.Layout(height="auto", width="100"))
g22[5,1] = subwell_c_reservoir
subwell_c_seed = widgets.Text(layout=widgets.Layout(height="auto", width="100"))
g22[5,2] = subwell_c_seed

g22[6,0:] = widgets.Button(description="subwell d", layout=widgets.Layout(height="auto", width="auto"), 
                            button_style="success")
g22[7,0] = widgets.Button(description="V(Protein) [nL]", layout=widgets.Layout(height="auto", width="auto"), 
                            button_style="success")
g22[7,1] = widgets.Button(description="V(Reservoir) [nL]", layout=widgets.Layout(height="auto", width="auto"), 
                            button_style="success")
g22[7,2] = widgets.Button(description="V(Seed) [nL]", layout=widgets.Layout(height="auto", width="auto"), 
                            button_style="success")

subwell_d_protein = widgets.Text(layout=widgets.Layout(height="auto", width="100"))
g22[8,0] = subwell_d_protein
subwell_d_reservoir = widgets.Text(layout=widgets.Layout(height="auto", width="100"))
g22[8,1] = subwell_d_reservoir
subwell_d_seed = widgets.Text(layout=widgets.Layout(height="auto", width="100"))
g22[8,2] = subwell_d_seed

#get_plate_from_db_button = widgets.Button(description='Get CrystalPlate from Database',
#                                          layout=widgets.Layout(height="auto", width="auto"), 
#                                           style= {'button_color':'gray'})
#get_plate_from_db_button.on_click(get_plate_from_db)
#grid2[10,0] = get_plate_from_db_button

save_plate_to_db_button = widgets.Button(description='Save CrystalPlate to Database',
                                          layout=widgets.Layout(height="auto", width="auto"), 
                                           style= {'button_color':'gray'})
save_plate_to_db_button.on_click(save_plate_to_db)
g22[10,0:] = save_plate_to_db_button                                                                          

In [15]:
#
# Functions for Inspect Plate tab
#

def change_next_crystal_image(b):
    global crystal_image_number
    crystal_image_number += 1
    if crystal_image_number > len(image_list):
        crystal_image_number = len(image_list)
    show_crystal_image(crystal_image_number)
        
def change_prev_crystal_image(b):
    global crystal_image_number
    crystal_image_number += -1
    if crystal_image_number < 0:
        crystal_image_number = 0
    show_crystal_image(crystal_image_number)

def show_crystal_image(n):
    global image_list
    crystal_image_progress.value = n
    out = widgets.Output()    
    i = image_list[n]
    tmp = i[i.rfind('/')+1:].replace('.png','')
    global well
    well = tmp[:3]
    global subwell
    subwell = tmp[3:4]
    barcode = select_crystal_barcode.value
    marked_crystal_id = barcode + '-' + well + subwell
    query = db.select([markedcrystalTable.columns.MarkedCrystal_ID.distinct()])
    ResultProxy = connection.execute(query)
    marked_crystals = [x[0] for x in ResultProxy.fetchall()]
    with cbook.get_sample_data(image_list[n]) as image_file:
        image = plt.imread(image_file)        
    if marked_crystal_id in marked_crystals:
        plt.figure(linewidth=20, edgecolor='green')
    plt.imshow(image)
    plt.axis("off")
#    plt.figure(linewidth=10)    
    plt.title(select_crystal_barcode.value + ' ' + well + subwell)
    with out:
        clear_output(wait=True)
        display(plt.show())
    vbox_cystal_image.children = [out]

def refresh_crystal_barcode(b):
    query = db.select([crystalplateTable.columns.CrystalPlate_Barcode.distinct()])
    ResultProxy = connection.execute(query)
    existing_crystalplates = [x[0] for x in ResultProxy.fetchall()]
    select_crystal_barcode.options = existing_crystalplates
    logger.info('updating barcode selection dropdown: ' + str(existing_crystalplates))

def load_crystal_images(b):
    global crystal_image_number
    crystal_image_number = 0
    barcode = select_crystal_barcode.value
    logger.info('trying to load crystal images from plate ' + barcode)
    global image_list
    image_list = []
    for img in sorted(glob.glob(os.path.join(crystal_image_folder,barcode,'*'))):
#        logger.info('loading ' + img)
        image_list.append(img)
    if image_list == []:
        logger.error('cannot find images for plate ' + barcode)

def update_marked_crystal_in_db(marked_crystal_id, barcode, well, subwell):
    query = db.select([markedcrystalTable.columns.MarkedCrystal_ID.distinct()])
    ResultProxy = connection.execute(query)
    marked_crystals = [x[0] for x in ResultProxy.fetchall()]

    if marked_crystal_id in marked_crystals:
        logger.warning('crystal already flagged for soaking; skipping...')
    else:
        logger.info('marking crystal for soaking in database: ' + marked_crystal_id)
        values_list = [{
            'MarkedCrystal_ID':      marked_crystal_id,
            'CrystalPlate_Barcode':  barcode,
            'CrystalPlate_Well':     well,
            'CrystalPlate_Subwell':  subwell
                }]  
        query = db.insert(markedcrystalTable) 
        ResultProxy = connection.execute(query,values_list)

        
        
def flag_for_soaking(b):
    barcode = select_crystal_barcode.value
    global well
    global subwell
    marked_crystal_id = barcode + '-' + well + subwell
    logger.info('flagging ' + well + subwell + ' from plate ' + barcode + ' for soaking')

    update_marked_crystal_in_db(marked_crystal_id, barcode, well, subwell)
#    query = db.select([markedcrystalTable.columns.MarkedCrystal_ID.distinct()])
#    ResultProxy = connection.execute(query)
#    marked_crystals = [x[0] for x in ResultProxy.fetchall()]
#
#    if marked_crystal_id in marked_crystals:
#        logger.warning('crystal already flagged for soaking; skipping...')
#    else:
#        logger.info('marking crystal for soaking in database')
#        values_list = [{
#            'MarkedCrystal_ID':      marked_crystal_id,
#            'CrystalPlate_Barcode':  barcode,
#            'CrystalPlate_Well':     well,
#            'CrystalPlate_Subwell':  subwell
#                }]  
#        query = db.insert(markedcrystalTable) 
#        ResultProxy = connection.execute(query,values_list)

    global crystal_image_number
    crystal_image_number += 1
    if crystal_image_number > len(image_list):
        crystal_image_number = len(image_list)
    show_crystal_image(crystal_image_number)

def show_marked_crystals(wellList):
    magnify = 2
    out = widgets.Output() 
    fig, ax = plt.subplots()
    plt.xlim(0,36 * magnify)
    plt.ylim(0,25 * magnify)
    ax.set_aspect(1)
    for c in misc.swiss_ci_layout():
        well = c[0]
        x = c[1] * magnify
        y = c[2] * magnify
        radius = 0.2 * magnify
        if c[3] == 'circle' and well in wellList:
            ax.add_artist(plt.Circle((x, y), radius, color='red'))
        if c[3] == 'circle' and not well in wellList:
            ax.add_artist(plt.Circle((x, y), radius, color='gray'))
        elif c[3] == 'rectangle':
            ax.add_patch(Rectangle((x-radius, y-radius), radius*2, radius*2,edgecolor='black',facecolor='none',lw=0.2))    
    plt.axis("off")
    with out:
        clear_output(wait=True)
        display(plt.show())
    vbox_cystal_image.children = [out]

    
def import_shifter_marked_crystals(b):
    clear_output()                                         # Button is deleted after it is clicked.
    root = Tk()
    root.withdraw()                                        # Hide the main window.
    root.call('wm', 'attributes', '.', '-topmost', True)   # Raise the root to the top of all windows.
    b.files = filedialog.askopenfilename(multiple=True,
                                         initialdir=os.path.join(workflow_folder,'1-inspect'), 
                                         title="Select file",
                                         filetypes=[("Text Files", "*.csv")])    # List of selected files will be set button's file attribute.
    
    wellList = []
    if os.path.isfile(b.files[0]):
        logger.info('loading ' + b.files[0])
        for line in open(b.files[0]):
            if line.startswith(';'):
                continue
            barcode = re.split(r'[ ,;]+', line)[1]
            row = re.split(r'[ ,;]+', line)[3]
            column = '0'*(2-len(re.split(r'[ ,;]+', line)[4])) + re.split(r'[ ,;]+', line)[4]
            subwell = re.split(r'[ ,;]+', line)[5]
            well = row + column
            marked_crystal_id = barcode + '-' + well + subwell
            update_marked_crystal_in_db(marked_crystal_id, barcode, well, subwell)
            wellList.append(well + subwell)
    else:
        logger.error('cannot read file ' + b.files[0])
    
    show_marked_crystals(wellList)

In [16]:
#
# Inspect Plate tab for import of shifter inspection
#

import_shifter_marked_crystals_button = widgets.Button(description='import marked crystals from shifter')
import_shifter_marked_crystals_button.on_click(import_shifter_marked_crystals)
#import_shifter_marked_crystals_button.on_click(show_marked_crystals)


vbox_cystal_image = widgets.VBox(children=[])



In [17]:
#
# Inspect Plate tab for inspection of crystal images
#


#g3 = widgets.GridspecLayout(2, 4)

#g3[0,0] = widgets.Button(description="Select Barcode", 
#                           layout=widgets.Layout(height="auto", width="auto"), style= {'button_color':'white'})
#select_crystal_barcode = widgets.Dropdown()
#g3[0,1] = select_crystal_barcode
#load_crystal_barcode_button = widgets.Button(description='Refresh Barcode List')
#load_crystal_barcode_button.on_click(refresh_crystal_barcode)
#g3[0,2] = load_crystal_barcode_button
#load_crystal_image_button = widgets.Button(description='Load Images')
#load_crystal_image_button.on_click(load_crystal_images)
#g3[0,3] = load_crystal_image_button



#global crystal_image_number
#crystal_image_number = 0

#global image_list
#image_list = []

#global well
#well = None

#global subwell
#subwell = None

#prev_image_button = widgets.Button(description='<<<')
#next_image_button = widgets.Button(description='>>>')

#prev_image_button.on_click(change_prev_crystal_image)
#next_image_button.on_click(change_next_crystal_image)

#flag_crystal_button = widgets.Button(description='Flag for Soaking', style= {'button_color':'green'})
#flag_crystal_button.on_click(flag_for_soaking)

#unflag_crystal_button = widgets.Button(description='Unflag', style= {'button_color':'red'})



#vbox_cystal_image = widgets.VBox(children=[])

##crystal_image_progress = IntProgress(min=0, max=len(image_list))
#crystal_image_progress = IntProgress(min=0, max=287)

#g32 = widgets.GridspecLayout(2, 4)

#import_shifter_marked_crystals_button = widgets.Button(description='import marked crystals from shifter')
#import_shifter_marked_crystals_button.on_click(import_shifter_marked_crystals)

#g32[0,0] = import_shifter_marked_crystals_button


In [18]:
#
# Functions for Soak Plate tab
#

def add_soakplate(b):
    l = []
    for opt in select_soakplate.options: l.append(opt)
    if soakplate_name.value not in l:
        logger.info('adding ' + soakplate_name.value + ' to soakplate dropdown')
        l.append(soakplate_name.value)
    else:
        logger.warning(soakplate_name.value + ' exists in soakplate dropdown')
    select_soakplate.options = l

def refresh_soakplate(b):
    query = db.select([soakplateTable.columns.SoakPlate_Name.distinct()])
    ResultProxy = connection.execute(query)
    existing_soakplates = [x[0] for x in ResultProxy.fetchall()]
    select_soakplate.options = existing_soakplates
    logger.info('updating soakplate dropdown: ' + str(existing_soakplates))

def refresh_libraryplate(b):
    query = db.select([compoundbatchTable.columns.CompoundPlate_Name.distinct()])
    ResultProxy = connection.execute(query)
    existing_compoundplates = [x[0] for x in ResultProxy.fetchall()]
    select_library_plate.options = existing_compoundplates
    logger.info('updating Library plate selection dropdown: ' + str(existing_compoundplates))
    
def save_soakplate_to_db(b):
    logger.info('saving soakplate information for ' + select_soakplate.value + ' to database')
    query = db.select([soakplateTable.columns.SoakPlate_Name.distinct()])
    ResultProxy = connection.execute(query)
    existing_soakplates = [x[0] for x in ResultProxy.fetchall()]   

    df_template = pd.read_csv(crystal_plate_template)
    for index, row in df_template.iterrows():
        well = df_template.at[index,'CrystalScreen_Well']

    query = db.select([compoundbatchTable.columns.CompoundPlate_Well, 
                       compoundbatchTable.columns.CompoundBatch_ID]).where(
                       compoundbatchTable.columns.CompoundPlate_Name == select_library_plate.value)
    ResultProxy = connection.execute(query)
    results = ResultProxy.fetchall()

    try:
        _soakplate_reservoir_volume = float(soakplate_reservoir_volume.value)
    except ValueError:
        _soakplate_reservoir_volume = 0.0
    
    try:
        _soakplate_compound_volume = float(soakplate_compound_volume.value)
    except ValueError:
        _soakplate_compound_volume = 0.0
    
    soakRows = []
    for r in results:
        well = r[0]
        column = well[0]
        row = well[1:3]
        cpd = r[1]
        soakplate_condition_id = select_soakplate.value + '-' + well
    
        if select_soakplate.value in existing_soakplates:
            logger.warning('soakplate ' + select_soakplate.value + ' exists in database; updating records...')
            query = db.update(soakplateTable).values(
                SoakPlate_Name = select_soakplate.value,
                SoakPlate_Well = well,
                SoakPlate_Subwell = 'a',
                CompoundBatch_ID = cpd,
                CompoundPlate_Name = select_library_plate.value,
                CrystalBuffer = soakplate_reservoir.value,
                CrystalBuffer_Vol = _soakplate_reservoir_volume,
                Compound_Vol = _soakplate_compound_volume
                ).where(soakplateTable.columns.SoakPlate_Condition_ID == soakplate_condition_id)
            results = connection.execute(query)
        else:
            logger.info('plate barcode ' + select_soakplate.value + ' does not exist in database; inserting records...')
            values_list = [{
                'SoakPlate_Condition_ID': soakplate_condition_id,
                'SoakPlate_Name':         select_soakplate.value,
                'SoakPlate_Well':         well,
                'SoakPlate_Subwell':      'a',
                'CompoundBatch_ID':       cpd,
                'CompoundPlate_Name':     select_library_plate.value,
                'CrystalBuffer':          soakplate_reservoir.value,
                'CrystalBuffer_Vol':      _soakplate_reservoir_volume,
                'Compound_Vol':           _soakplate_compound_volume
                  }]  
            query = db.insert(soakplateTable) 
            ResultProxy = connection.execute(query,values_list)

            
        soakRowDict = {
            'PlateType': 'SwissCI-MRC-3d',
            'PlateID': select_soakplate.value,
            'LocationShifter': 'AM',
            'PlateColumn': column,
            'PlateRow': row,
            'PositionSubWell': 'a',
            'ExternalComment': cpd
            }
        soakRows.append(soakRowDict)

    save_soakplate_csv_files(soakRows, select_soakplate.value)
            
            

def save_soakplate_csv_files(soakRows, soakPlate):     
    if os.path.isfile(os.path.join(workflow_folder,'2-soak', soakPlate + '.csv')):
        logger.warning('soakplate CSV file exists: ' + os.path.join(workflow_folder,'2-soak', soakPlate + '.csv'))
        now = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
        logger.info('moving exisiting soakplate CSV file to backup folder as: ' + soakPlate + '.csv.' + now)
        move(os.path.join(workflow_folder,'2-soak', soakPlate + '.csv'),os.path.join(workflow_folder,'2-soak', soakPlate + '.csv.' + now))
        
    fieldnames = misc.shifter_csv_header()
    with open(os.path.join(workflow_folder,'2-soak', soakPlate + '.csv'), 'w', encoding='UTF8', newline='') as f:
        writer = csv.DictWriter(f, fieldnames=fieldnames)
        writer.writeheader()
        writer.writerows(soakRows)        



In [19]:
#
# Soak Plate setup
#

g4 = widgets.GridspecLayout(10, 4)

g4[0,0] = widgets.Button(description="Enter New Soak Plate Name", 
                           layout=widgets.Layout(height="auto", width="auto"), style= {'button_color':'white'})
soakplate_name = widgets.Text(value='', layout=widgets.Layout(height="auto", width="100"))
g4[0,1] = soakplate_name
add_soakplate_name_button = widgets.Button(description='Add')
add_soakplate_name_button.on_click(add_soakplate)
g4[0,2] = add_soakplate_name_button

g4[1,0] = widgets.Button(description="Select Soak Plate", 
                           layout=widgets.Layout(height="auto", width="auto"), style= {'button_color':'white'})
select_soakplate = widgets.Dropdown()
g4[1,1] = select_soakplate
refresh_soakplate_button = widgets.Button(description='Refresh Soak Plate List')
refresh_soakplate_button.on_click(refresh_soakplate)
g4[1,2] = refresh_soakplate_button

g4[2,0] = widgets.Button(description="Select Library Plate", 
                           layout=widgets.Layout(height="auto", width="auto"), style= {'button_color':'white'})
select_library_plate = widgets.Dropdown()
g4[2,1] = select_library_plate
refresh_library_plate_button = widgets.Button(description='Refresh Library Plate List')
refresh_library_plate_button.on_click(refresh_libraryplate)
g4[2,2] = refresh_library_plate_button




g4[3,0] = widgets.Button(description="Reservoir Condition", 
                           layout=widgets.Layout(height="auto", width="auto"), style= {'button_color':'white'})
soakplate_reservoir = widgets.Text(value='', layout=widgets.Layout(height="auto", width="200"))
g4[3,1:] = soakplate_reservoir

g4[4,0] = widgets.Button(description="Reservoir Volume (\u03BCL)", 
                           layout=widgets.Layout(height="auto", width="auto"), style= {'button_color':'white'})
soakplate_reservoir_volume = widgets.Text(value='', layout=widgets.Layout(height="auto", width="200"))
g4[4,1:] = soakplate_reservoir_volume

g4[5,0] = widgets.Button(description="Compound Volume (\u03BCL)", 
                           layout=widgets.Layout(height="auto", width="auto"), style= {'button_color':'white'})
soakplate_compound_volume = widgets.Text(value='', layout=widgets.Layout(height="auto", width="200"))
g4[5,1] = soakplate_compound_volume

save_soakplate_button = widgets.Button(description='Save Soak Plate to DB',  style= {'button_color':'gray'})
save_soakplate_button.on_click(save_soakplate_to_db)
g4[6,0:] = save_soakplate_button


In [20]:
#def shifter_csv_header():
#    header = [
#        'PlateType',
#        'PlateID',
#        'LocationShifter',
#        'PlateRow',
#        'PlateColumn',
#        'PositionSubWell',
#        'Comment',
#        'CrystalID',
#        'TimeArrival',
#        'TimeDeparture',
#        'PickDuration',
#        'DestinationName',
#        'DestinationLocation',
#        'Barcode',
#        'ExternalComment'
#        ]
#    return header

In [21]:
#
# Functions for Crystal Soak tab
#

#def update_soak_setup_table(b):
#    # this will first update the SoakedCrystals table in case new MarkedCrystals or SoakPlates appeared    
#    query = db.select([soakedcrystalTable.columns.MarkedCrystal_ID.distinct()])
#    ResultProxy = connection.execute(query)
#    marked_crystals_soaked = [x[0] for x in ResultProxy.fetchall()]
#
#    query = db.select([soakedcrystalTable.columns.SoakPlate_Condition_ID.distinct()])
#    ResultProxy = connection.execute(query)
#    soak_condition_distinct = [x[0] for x in ResultProxy.fetchall()]
#
#
#    
##        
##    query = db.select([markedcrystalTable.columns.MarkedCrystal_ID.distinct()]).order_by(
##                                            markedcrystalTable.columns.MarkedCrystal_ID.asc())
##    ResultProxy = connection.execute(query)
##    marked_crystals_soaked = [x[0] for x in ResultProxy.fetchall()]
##    
##    query = db.select([soakplateTable.columns.SoakPlate_Condition_ID.distinct()]).order_by(
##                                            soakplateTable.columns.SoakPlate_Condition_ID.asc())
##    ResultProxy = connection.execute(query)
##    soak_condition_distinct = [x[0] for x in ResultProxy.fetchall()]
##
##    for item in result:
##        print(item)
##        if item[0] in marked_crystals_soaked:
##            result.remove(item)
##        if item[1] in soak_condition_distinct:
##            result.remove(item)
##    
##    marked_crystals_new = db.select([markedcrystalTable.columns.MarkedCrystal_ID.distinct()]).filter(
##        MarkedCrystal_ID.notin_(marked_crystals_soaked))
##    
##    
##    print('hallo')
#
## need another query here, where we obtain the same information as below, but from soakedcrystalTable
#
##    query = ...
#    
#    # note: rows should be coloured by batch number
#
#    query = db.select([markedcrystalTable.columns.CrystalPlate_Barcode,
#                         markedcrystalTable.columns.CrystalPlate_Well,
#                         markedcrystalTable.columns.CrystalPlate_Subwell]).where(
#                        markedcrystalTable.columns.MarkedCrystal_ID.notin_(marked_crystals_soaked)).order_by(
#                         markedcrystalTable.columns.CrystalPlate_Barcode.asc(), 
#                         markedcrystalTable.columns.CrystalPlate_Well.asc(),
#                         markedcrystalTable.columns.CrystalPlate_Subwell.asc())
#    ResultProxy = connection.execute(query)
#    result = ResultProxy.fetchall()
#    df1 = pd.read_sql_query(query, engine)
##    df1.loc[df1['SoakBatch_ID'] == 'null', 'SoakBatch_ID'] = '0'
##    df1['SoakBatch_ID'].replace([None], '0', inplace=True)
#
#    query = db.select([soakplateTable.columns.SoakPlate_Name,
#                         soakplateTable.columns.SoakPlate_Well,
#                         soakplateTable.columns.SoakPlate_Subwell,
#                         soakplateTable.columns.CompoundBatch_ID
#                      ]).where(
#                soakplateTable.columns.SoakPlate_Condition_ID.notin_(soak_condition_distinct)).order_by(
#                         soakplateTable.columns.SoakPlate_Well.asc(), 
#                         soakplateTable.columns.SoakPlate_Subwell.asc())
#    ResultProxy = connection.execute(query)
#    result = ResultProxy.fetchall()
#    df2 = pd.read_sql_query(query, engine)
##    print('==>',df2.at[0,'SoakBatch_ID'])
##    df2.loc[df2['SoakBatch_ID'] == None, 'SoakBatch_ID'] = '0'
##    df2['SoakBatch_ID'].replace([None], '0', inplace=True)
##    print('====>',df2.at[0,'SoakBatch_ID'])
##    print(df2['SoakBatch_ID'])
#    df = pd.concat([df1, df2], axis=1)
#    df['SoakBatch_ID'] = '0'
##    df = pd.merge(df1, df2, on="SoakBatch_ID")
#
##    soak_sheet.addCellHighlighter(TableDisplayCellHighlighter.getHeatmapHighlighter("CrystalPlate_Barcode", TableDisplayCellHighlighter.FULL_ROW))
#
#    for index, row in df.iterrows():
#        soak_sheet.updateCell(index,"CrystalPlate_Barcode",df.at[index,'CrystalPlate_Barcode'])
#        soak_sheet.values[index][1] = df.at[index,'CrystalPlate_Well']
#        soak_sheet.values[index][2] = df.at[index,'CrystalPlate_Subwell']
#        soak_sheet.values[index][3] = df.at[index,'SoakBatch_ID']
#        soak_sheet.values[index][4] = df.at[index,'SoakPlate_Name']
#        soak_sheet.values[index][5] = df.at[index,'SoakPlate_Well']
#        soak_sheet.values[index][6] = df.at[index,'SoakPlate_Subwell']
#        soak_sheet.values[index][7] = df.at[index,'CompoundBatch_ID']
#    soak_sheet.sendModel()


#def make_soak_batches(b):
##    batch = 0
#    current = ['nan','nan']
#    maxBatchList = []
##    foundZeroBatch = False
#    for index,i in enumerate(soak_sheet.values):
#        xtalPlate = i[0]
#        soakPlate = i[4]
#        if str(xtalPlate) == 'nan' or str(soakPlate) == 'nan':
#            break
#        try:
#            current_batch = int(i[3])
#            if current_batch not in maxBatchList:
#                maxBatchList.append(current_batch)
#        except ValueError:
#            # this will happen once the rows filled with "......." occur
#            break
##        print(str(xtalPlate), soakPlate)
#        if current_batch == 0:
#            if xtalPlate != current[0] or soakPlate != current[1]:
##            if not foundZeroBatch:
#                current = [xtalPlate, soakPlate]
#                foundZeroBatch = True
#                batch = max(maxBatchList) + 1
#                maxBatchList.append(batch)
##                print(batch)
##                soak_sheet.values[n][3] = str(batch)
##                soak_sheet.updateCell(index,"SoakBatch_ID",str(batch))
##            if xtalPlate != current[0] or soakPlate != current[1]:
##                current = [xtalPlate, soakPlate]
##                batch +=1
##                print(batch)
##                soak_sheet.values[n][3] = str(batch)
##                soak_sheet.updateCell(index,"SoakBatch_ID",str(batch))
#            else:
#                batch = max(maxBatchList)
#            soak_sheet.updateCell(index,"SoakBatch_ID",str(batch))
#        else:
#            continue
#    soak_sheet.sendModel()

#def write_batch_csv_files(xtalRows, soakRows, batch, xtalPlate, soakPlate):       
#    fieldnames = misc.shifter_csv_header()
#    with open(os.path.join(workflow_folder,'2-soak',str(batch) + '-xtal_batch-'+ xtalPlate + '.csv'), 'w', 
#              encoding='UTF8', newline='') as f:
#        writer = csv.DictWriter(f, fieldnames=fieldnames)
#        writer.writeheader()
#        writer.writerows(xtalRows)        
#    with open(os.path.join(workflow_folder,'2-soak',str(batch) + '-comp_batch-' + soakPlate + '.csv'), 'w', 
#              encoding='UTF8', newline='') as f:
#        writer = csv.DictWriter(f, fieldnames=fieldnames)
#        writer.writeheader()
#        writer.writerows(soakRows)        

    
#def save_soak_batches(b):
#    # every time a plate needs to be changed in the shifter, then a new batch is created
#    batchList = []
#    xtalRows = []
#    soakRows = []
#    xtalPlate_previous = None
#    soakPlate_previous = None
#    
#    firstBatch = True
#    
#    for index,i in enumerate(soak_sheet.values):
#        xtalPlate = i[0]
#        soakPlate = i[4]
#        if str(xtalPlate) == 'nan' or str(soakPlate) == 'nan':
#            if xtalRows != []:
#                write_batch_csv_files(xtalRows, soakRows, batch, xtalPlate_previous, soakPlate_previous)           
#            break
#        batch = i[3]
#        if batch not in batchList:
#            batchList.append(batch)
#            if xtalRows != []:
#                write_batch_csv_files(xtalRows, soakRows, batch, xtalPlate_previous, soakPlate_previous)           
#            xtalRows = []
#            soakRows = []
#
#        xtalPlate_previous = xtalPlate
#        soakPlate_previous = soakPlate
#        xtalPlate_Well = i[1]
#        xtalPlate_Row = xtalPlate_Well[0]
#        xtalPlate_Column = xtalPlate_Well[1:3]
#        xtalPlate_Subwell = i[2]        
#        soakPlate_Well = i[5]
#        soakPlate_Row = soakPlate_Well[0]
#        soakPlate_Column = soakPlate_Well[1:3]
#        soakPlate_Subwell = i[6]
#        compound_ID = i[7]
#        xtalRowDict = {
#            'PlateType': 'SwissCI-MRC-3d',
#            'PlateID': xtalPlate,
#            'LocationShifter': 'AM',
#            'PlateRow': xtalPlate_Row,
#            'PlateColumn': xtalPlate_Column,
#            'PositionSubWell': xtalPlate_Subwell,
#            'ExternalComment': compound_ID
#            }
#        xtalRows.append(xtalRowDict)
#        soakRowDict = {
#            'PlateType': 'SwissCI-MRC-3d',
#            'PlateID': soakPlate,
#            'LocationShifter': 'AM',
#            'PlateRow': soakPlate_Row,
#            'PlateColumn': soakPlate_Column,
#            'PositionSubWell': soakPlate_Subwell,
#            'ExternalComment': compound_ID
#            }
#        soakRows.append(soakRowDict)

        
def import_soaked_crystals(b):
    clear_output()                                         # Button is deleted after it is clicked.
    root = Tk()
    root.withdraw()                                        # Hide the main window.
    root.call('wm', 'attributes', '.', '-topmost', True)   # Raise the root to the top of all windows.
    b.files = filedialog.askopenfilename(multiple=True,
                                         initialdir=os.path.join(workflow_folder,'2-soak'), 
                                         title="Select file",
                                         filetypes=[("Text Files", "*xtal*.csv")])    # List of selected files will be set button's file attribute.
    
    if os.path.isfile(b.files[0]):
        logger.info('loading ' + b.files[0])
        for line in open(b.files[0]):
            if line.startswith(';'):
                continue
            xtal = re.split(r'[,;]+', line)[7]
            puck = re.split(r'[,;]+', line)[11]
            position = re.split(r'[,;]+', line)[12]
            
            timestamp = re.split(r'[,;]+', line)[9] 
            
            barcode = re.split(r'[,;]+', line)[1]
            row = re.split(r'[,;]+', line)[3]
            column = '0'*(2-len(re.split(r'[,;]+', line)[4])) + re.split(r'[,;]+', line)[4]
            subwell = re.split(r'[,;]+', line)[5]
            well = row + column

    else:
        logger.error('cannot read file ' + b.files[0])
    

def import_crystal_soak_csv(b):
    #
    # query transferred compounds
    query = db.select([soakplateTable.columns.SoakPlate.distinct()])
    ResultProxy = connection.execute(query)
    soakplates = [x[0] for x in ResultProxy.fetchall()]
    
    for soakPlate in soakplates:
        if os.path.isfile(os.path.join(workflow_folder,'2-soak', soakPlate + '.csv')):
            logger.info('found {0!s}.csv'.format(soakPlate))
            
            
        else:
            logger.warning('cannot find {0!s in {1!s}}'.format(soakPlate, os.path.join(workflow_folder,'2-soak')))
            
        
    
    # query soaked crystals
#    for csv in glob.glob soaked crystals
    
    # write csv file in mount folder if respective file does not exist

def update_crystal_soak_table(b):
    print('hallo')
    

In [22]:
#
# Crystal Soak
#

n_rows = 2000

#soak_sheet = sheet(rows=n_rows, columns=8, colu6mn_headers=[
#    'CrystalPlate_barcode',
#    'CrystalPlate_Well',
#    'CrystalPlate_Subwell',
#    'SoakPlate_Name',
#    'SoakPlate_Well',
#    'Compound_batch_ID'
#])
#soak_sheet.layout.height = '350px'   # adjust to monitor size
##cell(1,0,False)

headerList = [
    'CrystalPlate_Barcode', 
    'CrystalPlate_Well',
    'CrystalPlate_Subwell',
    'SoakBatch_ID',
    'SoakPlate_Name',
    'SoakPlate_Well',
    'SoakPlate_Subwell',
    'CompoundBatch_ID'
]

x = []

for i in range(1000):
    m = {}
    for j in range(len(headerList)):
        key = headerList[j]
        value = "............"     # cannot be space
        m[key] = value
    x.append(m)

#soak_sheet = TableDisplay(x)
#soak_sheet = TableDisplay([{
#'CrystalPlate_Barcode': '...',
#'CrystalPlate_Well': '...',
#'CrystalPlate_Subwell': '...',
#'SoakBatch_ID': '...',
#'SoakPlate_Name': '...',
#'SoakPlate_Well': '...',
#'SoakPlate_Subwell': '...',
#'CompoundBatch_ID': '...',
#    }])

#y = Label("Align Center", layout=Layout(display="flex", justify_content="center", width="30%", border="solid"))

g5 = widgets.GridspecLayout(10, 4)

#soak_sheet_vbox = VBox()
#soak_sheet_vbox.children = [soak_sheet]
#g5[0:8,0:3] = soak_sheet_vbox
#g5[0:8,0:3] = soak_sheet


#update_soak_setup_table_button = widgets.Button(description='Load')
#update_soak_setup_table_button.on_click(update_soak_setup_table)
#g5[0,3] = update_soak_setup_table_button

import_crystal_soak_csv_button = widgets.Button(description='Import CSVs')
import_crystal_soak_csv_button.on_click(import_crystal_soak_csv)
g5[0,3] = import_crystal_soak_csv_button

#make_soak_batches_button = widgets.Button(description='generate batches')
#make_soak_batches_button.on_click(make_soak_batches)
#g5[1,3] = make_soak_batches_button

#import_soaked_crystals_button = widgets.Button(description='Import soaked crystals')
#import_soaked_crystals_button.on_click(import_soaked_crystals)
#g5[1,3] = import_soaked_crystals_button

update_crystal_soak_table_button = widgets.Button(description='Update table')
update_crystal_soak_table_button.on_click(update_crystal_soak_table)
g5[1,3] = update_crystal_soak_table_button

#save_soak_batches_button = widgets.Button(description='Save batches')
#save_soak_batches_button.on_click(save_soak_batches)
#g5[2,3] = save_soak_batches_button

#import_soaked_crystals_button = widgets.Button(description='Import soaked crystals')
#import_soaked_crystals_button.on_click(import_soaked_crystals)
#g5[3,3] = import_soaked_crystals_button

#b3 = widgets.Button(description='Delete CP')
#g5[2,3] = b3
#b3 = widgets.Button(description='Save Batch')
#g5[3,3] = b3
#
#g5[4,3] = Label("Align Center", layout=Layout(display="flex", justify_content="center"))

#g

In [23]:
#
# Functions for Mounted Crystals tab
#

def import_mounted_crystals(b):
    clear_output()                                         # Button is deleted after it is clicked.
    root = Tk()
    root.withdraw()                                        # Hide the main window.
    root.call('wm', 'attributes', '.', '-topmost', True)   # Raise the root to the top of all windows.
    b.files = filedialog.askopenfilename(multiple=True,
                                         initialdir=os.path.join(workflow_folder,'3-mount'), 
                                         title="Select file",
                                         filetypes=[("Text Files", "*.csv")])    # List of selected files will be set button's file attribute.
    
    if os.path.isfile(b.files[0]):
        logger.info('loading ' + b.files[0])
        for line in open(b.files[0]):
            if line.startswith(';'):
                continue
            xtal = re.split(r'[,;]+', line)[7]
            puck = re.split(r'[,;]+', line)[11]
            position = re.split(r'[,;]+', line)[12]
            
            timestamp = re.split(r'[,;]+', line)[9] 
            
            barcode = re.split(r'[,;]+', line)[1]
            row = re.split(r'[,;]+', line)[3]
            column = '0'*(2-len(re.split(r'[,;]+', line)[4])) + re.split(r'[,;]+', line)[4]
            subwell = re.split(r'[,;]+', line)[5]
            well = row + column
            logger.info('{0!s} {1!s}'.format(xtal, timestamp))

    else:
        logger.error('cannot read file ' + b.files[0])

        

def update_db_with_manually_mounted_crystals(df):
    logger.info('here')
    query = db.select([mountedcrystalTable.columns.Manual_Crystal_ID.distinct()])
    ResultProxy = connection.execute(query)
    existing_manually_mounted_crystals = [x[0] for x in ResultProxy.fetchall()]
    logger.info('---> ' + str(existing_manually_mounted_crystals))   
    
    proteinacronym = None
    
    # latest crystal ID
    query = db.select([mountedcrystalTable.columns.Crystal_ID.distinct()]).order_by(mountedcrystalTable.columns.Crystal_ID.desc()).limit(1)
    ResultProxy = connection.execute(query)
    last_crystal_id = ResultProxy.fetchall()[0][0]
    proteinacronym = last_crystal_id.split('-')[0]
    logger.info('protein acronym is ' + proteinacronym)
    next_crystal_number = int(last_crystal_id.split('-')[1].replace('x','')) + 1
                                                                                
    if proteinacronym is None:
        if protein_acronym.value.replace(' ','') == '':
            logger.error('Please define protein acronym')
        
    for index, row in df.iterrows():
        Manual_Crystal_ID = df.at[index,'Manual_Crystal_ID']
        if Manual_Crystal_ID in existing_manually_mounted_crystals:
            # update
            print('hallo')
        
        else:
            Crystal_ID = protein_acronym + '-x' + '0'*(4-len(str(next_crystal_number))) + str(next_crystal_number)
            next_crystal_number += 1

#
#        
#        if CrystalScreen_Name in existing_crystalscreens:
#            logger.warning('crystal screen exists in database; updating records: {0!s} - {1!s}'.format(well, condition))
#            query = db.update(crystalscreenTable).values(
#                CrystalScreen_Condition = condition).where(
#                crystalscreenTable.columns.CrystalScreen_ID == screen_id)
#            results = connection.execute(query)
#        else:
#            logger.info('crystal screen does not exist in database; inserting records: {0!s} - {1!s}'.format(well, condition))
#            values_list = [{
#                'CrystalScreen_Condition': condition,
#                'CrystalScreen_Well':      well,
#                'CrystalScreen_Name':      CrystalScreen_Name,
#                'CrystalScreen_ID':        screen_id
#            }]  
#            query = db.insert(crystalscreenTable) 
#            ResultProxy = connection.execute(query,values_list)

        
        
        
def import_manually_mounted_crystals(b):
    clear_output()                                         # Button is deleted after it is clicked.
    root = Tk()
    root.withdraw()                                        # Hide the main window.
    root.call('wm', 'attributes', '.', '-topmost', True)   # Raise the root to the top of all windows.
    try:
        b.files = filedialog.askopenfilename(multiple=True,
                                         initialdir=os.path.join(workflow_folder,'4-mount-manual'), 
                                         title="Select file",
                                         filetypes=[("Text Files", "*.csv")])    # List of selected files will be set button's file attribute.
    
        if os.path.isfile(b.files[0]):
            logger.info('reading CSV file of manually mounted crystals from CSV file: ' + b.files[0])
            df = pd.read_csv(b.files[0])
            logger.info('read df')
            update_db_with_manually_mounted_crystals(df)
            logger.info('rgrr')
    
    except TypeError:
        logger.error('Please select project directory')


def save_template_manually_mounted_crystals(b):
    print('hallo')

In [24]:
#
# Mounted Crystals tab
#

n_rows_mounted_crystals = 1000

headerList_mounted_crystals = [
    'Crystal_ID', 
    'CompoundBatch_ID',
    'Pin_Barcode',
    'Puck_Position',
    'Puck_Name'
]

x = []

for i in range(n_rows_mounted_crystals):
    m = {}
    for j in range(len(headerList_mounted_crystals)):
        key = headerList_mounted_crystals[j]
        value = "............"     # cannot be space
        m[key] = value
    x.append(m)

#mounted_crystals_sheet = TableDisplay(x)

g6 = widgets.GridspecLayout(10, 4)

#soak_sheet_vbox = VBox()
#soak_sheet_vbox.children = [soak_sheet]
#g5[0:8,0:3] = soak_sheet_vbox
#g6[0:8,0:3] = mounted_crystals_sheet

import_mounted_crystals_button = widgets.Button(description='Import from Shifter')
import_mounted_crystals_button.on_click(import_mounted_crystals)
g6[0,3] = import_mounted_crystals_button

import_manually_mounted_crystals_button = widgets.Button(description='Import manual')
import_manually_mounted_crystals_button.on_click(import_manually_mounted_crystals)
g6[1,3] = import_manually_mounted_crystals_button

save_template_manually_mounted_crystals_button = widgets.Button(description='Save manual template')
save_template_manually_mounted_crystals_button.on_click(save_template_manually_mounted_crystals)
g6[2,3] = save_template_manually_mounted_crystals_button

export_csv_for_exi_button = widgets.Button(description='Export CSV for EXI')
#export_csv_for_exi_button.on_click(export_csv_for_exi)
g6[3,3] = export_csv_for_exi_button



In [25]:
%%html
<style>
.warning{
    background-color:red;
}
.all_good{
    background-color:white;
}
.info{
    background-color:yellow;
}
</style>

In [26]:
#
# tab widget
#

#tab0 = VBox(children=[Label(value = r'\(\color{green} {' + misc.project_description_note() + '}\)' ), g0])
tab0_info = widgets.HTML(value = f"<b><font color='black'>{misc.project_description_note()}</b>")
tab0_info.add_class("info")
tab0 = VBox(children=[tab0_info, g0])
tab0.add_class("warning")
#tab1 = VBox(children=[g1, screen_sheet,save_screen_to_db_button, crystal_screen_progress ])
#HBox([screen_sheet], layout={'width': '250px'})
tab1_info = widgets.HTML(value = f"<b><font color='black'>{misc.crystal_screen_note()}</b>")
tab1_info.add_class("info")
tab1 = VBox(children=[tab1_info, g1, HBox([screen_sheet], layout={'height': '250px'}),
                      save_screen_to_db_button, crystal_screen_progress ])
#tab1 = VBox(children=[g1, 
#                      save_screen_to_db_button, crystal_screen_progress ])

tab2 = VBox(children=[g21, g22])

#tab3 = VBox(children=[g3,
#                      HBox(children=[prev_image_button,next_image_button]),
#                      HBox(children=[flag_crystal_button,unflag_crystal_button]),
#                      vbox_cystal_image,crystal_image_progress,
#                      g32])

tab3 = VBox(children=[import_shifter_marked_crystals_button,
                      vbox_cystal_image])

tab4 = VBox(children=[g4])

tab5 = VBox(children=[g5])

tab6 = VBox(children=[g6])

tab = widgets.Tab(children=[tab0, tab1, tab2, tab3, tab4, tab5, tab6])
tab.set_title(0, 'Project Description')
tab.set_title(1, 'Crystal Screen')
tab.set_title(2, 'Crystal Plate')
tab.set_title(3, 'Inspect Plate')
tab.set_title(4, 'Soak Plate')
tab.set_title(5, 'Crystal Soak')
tab.set_title(6, 'Mounted Crystals')


VBox(children=[tab])


In [27]:
#
# logger output
#

handler.show_logs()

Output(layout=Layout(border='1px solid black', height='160px', width='100%'), outputs=({'name': 'stdout', 'out…

In [28]:
query = db.select([mountedcrystalTable.columns.Crystal_ID.distinct()]).order_by(mountedcrystalTable.columns.Crystal_ID.desc()).limit(1)
ResultProxy = connection.execute(query)
last_crystal_id = ResultProxy.fetchall()[0][0]
print(last_crystal_id)
n = int(last_crystal_id.split('-')[1].replace('x','')) + 1
print(n)

AttributeError: 'NoneType' object has no attribute 'columns'